In [1]:
from math import floor, ceil, sqrt
import numpy as np
import pandas as pd
from scipy import stats as st
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
import warnings

In [2]:
pd.options.display.max_columns = None

In [3]:
primes = []
prime_factor_dict = {}

In [4]:
def find_ring(n):
    segment = (n - 1) / 8
    triangle_root = (sqrt(8 * segment + 1) - 1) / 2
    ring = np.int64(ceil(triangle_root))
    return ring

In [5]:
def quarter_and_postition(n, ring):
    quarter_length = ring * 2
    ring_length = quarter_length * 4
    previous_ring_limit = (((ring * (ring - 1)) / 2) * 8) + 1
    adjusted_number = n - previous_ring_limit - 1
    ring_incriment = 1 / (ring * 8)
    ring_progress = ((adjusted_number + 1) / ring_length)
    ring_quarter = floor(adjusted_number / quarter_length)
    quarter_position = adjusted_number - (ring_quarter * quarter_length)
    return ring_quarter, quarter_position, adjusted_number, ring_progress, ring_incriment

In [6]:
def find_coordinates(ring, ring_quarter, quarter_position):
    base_quarter = (((ring_quarter + 1) * 2) - 5)
    mirror_modifier = base_quarter / abs(base_quarter) * -1 # 1, 1 ,-1 ,-1
    coordinate_modifier_a = ((ring_quarter) % 2) * mirror_modifier # 0, 1, 0, -1
    coordinate_modifier_b = ((ring_quarter + 1) % 2) * mirror_modifier # 1, 0, -1, 0
    quarter_position_modifier = (abs(base_quarter) - 2) # 1, -1, -1, 1
    quarter_position_shift = ring - 1
    adjusted_quarter_position = (quarter_position - quarter_position_shift) * quarter_position_modifier
    x1 = ring * coordinate_modifier_a
    y1 = ring * coordinate_modifier_b
    x2 = adjusted_quarter_position * abs(coordinate_modifier_b)
    y2 = adjusted_quarter_position * abs(coordinate_modifier_a)
    x = x1 + x2
    y = y1 + y2
    return x, y

In [7]:
def get_coordinates(data):
    n = data['number']
    if n == 1:
        x = 0
        y = 0
        ring = 0
        ring_progress = 1
        ring_incriment = 1
    else:
        ring = find_ring(n)
        ring_quarter, quarter_position, adjusted_number, ring_progress, ring_incriment = quarter_and_postition(n, ring)
        x, y = find_coordinates(ring, ring_quarter, quarter_position)
    return x, y, ring, ring_incriment, ring_progress

In [8]:
def check_prime(data):
    n = np.int64(data['number'])
    digits = [np.int64(digit) for digit in str(n)]
    digits_length = len(digits)
    first_digit = digits[0]
    last_digit = digits[-1]
    mean_digit = np.mean(digits)
    median_digit = np.median(digits)
    mode_digit, mode_count = st.mode(digits)
    prime = 1
    lpf= 1
    determining_prime = 1
    if n == 1:
        prime = 0
    elif n in [2, 3, 5]:
        prime = 1
        if n == 2:
            determining_prime = 2
        elif n == 3:
            determining_prime = 2
        else:
            determining_prime = 3
    elif last_digit not in [1, 3, 7, 9]:
        prime = 0
        if last_digit == 5:
            lpf = 5
            determining_prime = 5
        else:
            lpf = 2
            determining_prime = 2
    else:
        for p in primes:
            determining_prime = p
            if p > n / p:
                break
            elif n % p == 0:
                prime = 0
                lpf = p
                break
    if bool(prime):
        primes.append(n)
    return prime, determining_prime, lpf, digits, first_digit, last_digit, digits_length, mean_digit, median_digit, mode_digit, mode_count

In [9]:
def factorize(n, lpf, prime_factors, df):
    prime_factors.append(np.int64(lpf))
    new_n = n / lpf
    new_n_prime = df.loc[df['number'] == new_n, 'prime'].values[0]
    new_n_lpf = df.loc[df['number'] == new_n, 'lpf'].values[0]
    new_n_factors = prime_factor_dict[new_n]
    if new_n_prime == 0:
        if len(new_n_factors) > 0:
            prime_factors = prime_factors + new_n_factors
        else:
            prime_factors = factorize(new_n, new_n_lpf, prime_factors)
    else:
        prime_factors.append(np.int64(new_n))
    return prime_factors

In [10]:
def get_prime_factors(data, df):
    n = data['number']
    lpf = data['lpf']
    prime = data['prime']
    gpf = n if prime == 1 or n == 1 else 0
    prime_factors = []
    if gpf == 0:
        prime_factors = factorize(n, lpf, prime_factors, df)
        gpf = prime_factors[-1]
    else:
        prime_factors = [lpf, gpf]
    prime_factor_dict[n] = prime_factors
    factor_count = len(prime_factors)
    factor_mean = np.mean(prime_factors)
    factor_median = np.median(prime_factors)
    factor_mode, factor_mode_count = st.mode(prime_factors)
    return gpf, prime_factors, factor_count, factor_mean, factor_median, factor_mode, factor_mode_count